# Minimal Data Salesforce Data Set
> get pertinent data for contacts 4708 119430 119431

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from doubledot import *

In [139]:
from nbdev.showdoc import *
import requests
import json
import jmespath as jp
import re
from time import sleep
from fastcore.basics import patch
import fileinput
import pandas as pd
import os
import io
from fastcore.test import test_eq
import glob
import time
import random
from pyisemail import is_email

In [38]:
# used this to find members with 4708 contactId ( yes, there was an easier way but not faster given time and brain capacity)
atms.retrieve_and_clean('memberships', since_date='1995-01-01', max_rows=20000)

download dir is:  /Users/josephmann/Documents/Github/doubledot/atms_download/mlk31g16
resp_d = self.get_telus_data(memberships,offset=0, count= 1000, since_date=1995-01-01)
ATMS_api.get_telus_data: since_date is 1995-01-01
http://crm-api-telus.atmsplus.com/api/memberships/lastupdate?count=1000&offset=0&updateDate=1995-01-01
resp_d.keys():  dict_keys(['response', 'done'])
done: False, resp_d.json() : <Response [200]>
resp_d = self.get_telus_data(memberships,offset=1000, count= 1000, since_date=1995-01-01)
ATMS_api.get_telus_data: since_date is 1995-01-01
http://crm-api-telus.atmsplus.com/api/memberships/lastupdate?count=1000&offset=1000&updateDate=1995-01-01
resp_d.keys():  dict_keys(['response', 'done'])
done: False, resp_d.json() : <Response [200]>
resp_d = self.get_telus_data(memberships,offset=2000, count= 1000, since_date=1995-01-01)
ATMS_api.get_telus_data: since_date is 1995-01-01
http://crm-api-telus.atmsplus.com/api/memberships/lastupdate?count=1000&offset=2000&updateDate=1995-

# why does saleId__c = 889832 not have any Saledetails or Tickets pointing at it?



## Problems...